# Introduction

This notebook illustrates the relationship between power consumption and CPU load <br />
of a laptop computer. Sample power consumption and CPU load data is collected from <br />
a Macbook Pro having a Core i5 2.9 GHZ processor powered by MacOS Seirra. <br />

----
Tools used: 

**psutil**: to gather CPU load at user-defined sampling intervals

**MacOS SystemProfiler**: to collect battery discharge info. That is, 
                          voltage and current readings.
                          
**pandas and numpy**: to process the resulting data

**bokeh**: for plotting

In [1]:
import psutil
import subprocess
import time
import pandas as pd
import numpy as np

In [2]:
def cpuLoadPower(samples, sampling_interval, interval=None):
    cpu_load = []

    for sample in range(samples):
        subprocess.check_call("./power.sh", shell=True)
        cpu_load.append(psutil.cpu_percent(interval))
        time.sleep(sampling_interval)
    return cpu_load
    

In [ ]:
cpu_load = cpuLoadPower(1000, 3) # CPU load and Power consumption logging with 5 sec sampling interval

data = pd.read_csv('volt-amp.txt', sep='\t',header=None)
data.rename(columns={0:'Time', 1:'mV', 2: 'mA', 3:'mAh'}, inplace=True)
data['CPU'] = cpu_load
data['Power'] = data.mV * data.mA * -1e-6   # Power = V * A
data.Time = data.Time - data.Time[0]  # relative to the beginning time of sampling 


In [24]:
MAX_CHARGE = 6121 # To determine the maximum charge capacity run the System profiler 
data['Battery'] = data.mAh/MAX_CHARGE

In [25]:
data.head(5)

,Time,mV,mA,mAh,CPU,Power,Battery
0,0,12718,-980,5649,17.8,12.46364,0.922888
1,4,12718,-980,5649,28.9,12.46364,0.922888
2,7,12718,-980,5649,33.9,12.46364,0.922888
3,11,12718,-980,5649,33.8,12.46364,0.922888
4,15,12718,-980,5649,38.3,12.46364,0.922888


In [26]:
data.tail(5)

,Time,mV,mA,mAh,CPU,Power,Battery
995,3670,11918,-1464,4335,18.8,17.447952,0.708218
996,3673,11918,-1464,4335,22.3,17.447952,0.708218
997,3677,11918,-1464,4335,17.7,17.447952,0.708218
998,3680,11918,-1464,4335,22.5,17.447952,0.708218
999,3684,11913,-946,4347,20.1,11.269698,0.710178


In [9]:
data.to_csv("CPULoad_vs_Power.csv")

In [10]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [11]:
output_notebook()

Loading BokehJS ...

In [12]:
from bokeh.models import LinearAxis, Range1d

In [59]:
p = figure(plot_width=700, plot_height=400, toolbar_location="above")

p.line(data['Time'], data['CPU'], color="firebrick", legend='CPU load')

p.extra_y_ranges = {"foo": Range1d(start=0, end=35)}
p.line(data['Time'], data['Power'], color="navy", y_range_name="foo", legend='Power Consumption')

p.add_layout(LinearAxis(y_range_name="foo"), 'right')


p.xaxis.axis_label = "Time (sec)"

p.yaxis[0].axis_label = "CPU load (%)"
p.yaxis[0].axis_label_text_color = "firebrick"
p.yaxis[0].major_label_text_color = "firebrick"
p.yaxis[0].major_tick_line_color = 'firebrick'
p.yaxis[0].minor_tick_line_color = 'firebrick'
p.yaxis[0].axis_line_color = 'firebrick'

p.yaxis[1].axis_label = "Power (Watt)"
p.yaxis[1].axis_label_text_color = "navy"
p.yaxis[1].major_label_text_color = "navy"
p.yaxis[1].major_tick_line_color = 'navy'
p.yaxis[1].minor_tick_line_color = 'navy'
p.yaxis[1].axis_line_color = 'navy'

p.legend.location = "top_center"

p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.6
p.background_fill_color = "whitesmoke"
show(p)

The above plot shows that whenever CPU load hikes so does power consumption and vise versa.
Interestingly, there seems to be a consistent time gap between the change in CPU load and 
power consumption. 


In [64]:
fit = np.polyfit(data['CPU'], data['Power'], 1)
fn = np.poly1d(fit)

p = figure(plot_width=600, plot_height=400, toolbar_location="above")

p.circle(data.CPU, data.Power, color=None,line_color="firebrick")
p.line(data.CPU, fn(data.CPU), color="firebrick", legend="Power = 0.30 * CPU_Load + 6.09")

p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.6

p.xaxis.axis_label = "CPU LOAD (%)"
p.yaxis.axis_label = "POWER (Watt)"
p.legend.location = "top_left"
p.background_fill_color = "whitesmoke"
show(p)

The linear line in the above plot shows that the power consumption is about $11.94$ Watts <br /> 
even when the computer is idle. Of course, the CPU is not the only component that consumes power.<br />
Power consumption ranges between $12$ and $35$ Watts as the CPU load goes from $0$ to $100\%$.